In [ ]:
import sys
import re
import sqlite3
from tqdm.auto import tqdm
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
conn = sqlite3.connect('youngtyp.db')
cur = conn.cursor()

In [ ]:
# создаю пустую базу

cur.execute("""
CREATE TABLE IF NOT EXISTS texts
(id_text text, text text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS meta
(id_text text, author text, topic text, year int, affilation text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS sents 
(id_text text, id_sent int, sent text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS words
(id_text text, id_sent int, id_word int, word text collate nocase, lemma text, pos text)
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS words_right
(id_text text, id_sent int, id_word_right int, word_right text collate nocase, lemma_right text, pos_right text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS words_rright
(id_text text, id_sent int, id_word_rright int, word_rright text collate nocase, lemma_rright text, pos_rright text)
""")


conn.commit()  # создаю базу

In [ ]:
!wget -r 'https://raw.githubusercontent.com/hideousmaiden/youngtyp_corpora/sasha/theses.json'

--2022-10-23 18:45:44--  https://raw.githubusercontent.com/hideousmaiden/youngtyp_corpora/sasha/theses.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3733358 (3.6M) [text/plain]
Saving to: ‘raw.githubusercontent.com/hideousmaiden/youngtyp_corpora/sasha/theses.json’

raw.githubuserconte 100%[===================>]   3.56M  --.-KB/s    in 0.07s   

2022-10-23 18:45:44 (51.6 MB/s) - ‘raw.githubusercontent.com/hideousmaiden/youngtyp_corpora/sasha/theses.json’ saved [3733358/3733358]

FINISHED --2022-10-23 18:45:44--
Total wall clock time: 0.1s
Downloaded: 1 files, 3.6M in 0.07s (51.6 MB/s)


In [ ]:
with open('raw.githubusercontent.com/hideousmaiden/youngtyp_corpora/sasha/theses.json', 'r', encoding="utf-8") as f:
  bodies = json.load(f)

In [ ]:
# writing down raw text and sentences
for t_id in bodies.keys():
    cur.execute("""INSERT INTO texts
                (id_text, text) 
                VALUES (?, ?)""", (t_id, bodies[t_id]['text']))
    cur.execute("""INSERT INTO meta
                (id_text, author, topic, year, affilation) 
                VALUES (?, ?, ?, ?, ?)""", (t_id,
                                         bodies[t_id]['author'], bodies[t_id]['title'],
                                         bodies[t_id]['year'], bodies[t_id]['affilation']))
    sents = sent_tokenize(bodies[t_id]['text'])
    for spair in enumerate(sents):
      cur.execute("""INSERT INTO sents
                (id_text, id_sent, sent) 
                VALUES (?, ?, ?)""", (t_id, *spair))
conn.commit()

###  Парсинг

In [ ]:
#!pip install deeppavlov

In [ ]:
import deeppavlov
#!python -m deeppavlov install morpho_ru_syntagrus_pymorphy

In [ ]:
from deeppavlov import build_model, configs

#model = build_model(configs.morpho_tagger.UD2_0.morpho_ru_syntagrus_pymorphy, download=True)

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
# writing down words

cur.execute("""
    SELECT id_text, id_sent, sent
    FROM sents""")
sents = cur.fetchall()
for one in sents:
#    tags = list(filter(lambda x: len(x)>2, tags))
    words = word_tokenize(one[-1])
    words=[word for word in words if word.isalpha()]
    if len(words) == 0:
      continue
    tags = model(words)
    pos = list(map(lambda x: re.split('\t',x)[2] if len(x)>2 else x, tags))
    lems = list(map(lambda x: morph.parse(x)[0].normal_form, words))
    for wrd in zip(range(len(pos)), words, lems, pos):
        cur.execute("""INSERT INTO words
                (id_text, id_sent, id_word, word, lemma, pos) 
                VALUES (?, ?, ?, ?, ?, ?)""", (one[0], one[1], *wrd))
        cur.execute("""INSERT INTO words_right
                (id_text, id_sent, id_word_right, word_right, lemma_right, pos_right) 
                VALUES (?, ?, ?, ?, ?, ?)""", (one[0], one[1], *wrd))
        cur.execute("""INSERT INTO words_rright
                (id_text, id_sent, id_word_rright, word_rright, lemma_rright, pos_rright) 
                VALUES (?, ?, ?, ?, ?, ?)""", (one[0], one[1], *wrd))
conn.commit()

Так как в задании эксплицитно указано, что в поиске будет до трёх слов, поиск нам показалось удобным осуществлять по таблице, где для каждого слова в отдельных колонках указаны его правые соседи и их свойства.



> беда: SQL не может быть case-insensitive для русского

In [ ]:
def process_query(query, limit=10):
  """
  process cql query

  args:
    query: str,
  returns:
    result: list of dict, list of dictionaries containing found results
      pattern: str, found pattern
      id_sent: int, 
      id_text: int,
      id_word: int
      (last_word_id: int)
  """
# токены разделены словами
  parts = query.split(' ')
  # условия разделены плюсами
  conds = list(map(lambda x: x.split('+'), parts))
  pos_tr = {0:'pos',1:'pos_right', 2:'pos_rright'}
  lemma_tr = {0:'lemma',1:'lemma_right', 2:'lemma_rright'}
  word_tr = {0:'word',1:'word_right', 2:'word_rright'}
  que = []
  ask = {1:'''SELECT words.id_text, words.id_sent, words.id_word, words.word FROM words
  JOIN words_right ON words_right.id_sent=words.id_sent AND words_right.id_text=words.id_text AND id_word_right = (id_word + 1)
  JOIN words_rright ON words_rright.id_sent=words.id_sent AND words_rright.id_text=words.id_text AND id_word_rright = (id_word + 2)
  WHERE {} COLLATE NOCASE
  LIMIT {};''', 2:'''SELECT words.id_text, words.id_sent, words.id_word, words.word, id_word_right, word_right FROM words
  JOIN words_right ON words_right.id_sent=words.id_sent AND words_right.id_text=words.id_text AND id_word_right = (id_word + 1)
  JOIN words_rright ON words_rright.id_sent=words.id_sent AND words_rright.id_text=words.id_text AND id_word_rright = (id_word + 2)
  WHERE {} COLLATE NOCASE
  LIMIT {}
  ;''', 3:'''SELECT words.id_text, words.id_sent, words.id_word, words.word, id_word_rright, word_rright FROM words
  JOIN words_right ON words_right.id_sent=words.id_sent AND words_right.id_text=words.id_text AND id_word_right = (id_word + 1)
  JOIN words_rright ON words_rright.id_sent=words.id_sent AND words_rright.id_text=words.id_text AND id_word_rright = (id_word + 2)
  WHERE {} COLLATE NOCASE
  LIMIT {}'''}
  # по словам
  for nn, ques in enumerate(conds):
    for cond in ques:
      #все условия для одного слова
      if re.match('[A-Z]+', cond):
          
          que.append('{} = "{}"'.format(pos_tr[nn], cond))
      elif re.match(r'".+"', cond):
          que.append('(({} = {}) '.format(word_tr[nn], cond) + 'OR ({} = {}) '.format(word_tr[nn], cond.lower()) + 'OR ({} = {}))'.format(word_tr[nn], cond.title()))
          #que.append('{} = {}'.format(word_tr[nn], cond))
      else:
          que.append('{} = "{}"'.format(lemma_tr[nn], morph.parse(cond)[0].normal_form))
  big_que = ' AND '.join(que)
  ask = ask[len(conds)].format(big_que, limit)
  cur.execute(ask)
  done = set(cur.fetchall())
  result = []
  for res in list(done):
    dd = dict()
    dd['pattern'] = ' '.join(list(filter(lambda x: str(x).isalpha(), res)))
    dd['id_sent'] = res[1]
    dd['id_text'] = res[0]
    dd['id_word'] = res[2]
    if len(res) > 4:
      dd['last_word_id'] = res[-2]
    result.append(dd)
  return result

In [ ]:
process_query("ADP NOUN")

[{'pattern': 'на предложения',
  'id_sent': 3,
  'id_text': 0,
  'id_word': 8,
  'last_word_id': 9},
 {'pattern': 'В описании',
  'id_sent': 0,
  'id_text': 1,
  'id_word': 0,
  'last_word_id': 1},
 {'pattern': 'на библиотеку',
  'id_sent': 1,
  'id_text': 1,
  'id_word': 3,
  'last_word_id': 4},
 {'pattern': 'из текстов',
  'id_sent': 1,
  'id_text': 0,
  'id_word': 2,
  'last_word_id': 3}]